In [1]:
import init
import json
import pymongo
import os
import re

from llm import AliyunChat, split_plaintext_into_trunks
import tqdm

In [2]:
SYSTEM_PROMPT = """You are a professional information extraction assistant specialized in parsing Wikipedia biography pages.

Your task is to extract structured educational background from unstructured text and infoboxes. Focus on identifying undergraduate, graduate (master's), and doctoral (PhD) degrees. Your output must strictly follow the expected JSON format."""

def generate_education_prompt(trunk: str, infobox: str = "") -> str:
    """
    Generate the user prompt for educational background extraction.
    
    Args:
        trunk (str): A chunk of the plaintext.
        infobox (str): Optional infobox content from the Wikipedia page.
    
    Returns:
        str: The prompt for the LLM.
    """
    infobox_prompt = ""
    if infobox:
        infobox_prompt = f"\nInfobox:\n{infobox}\n\n"
        
    prompt = f"""Extract the person's educational background from the following text and infobox.

Return a JSON list of dictionaries. Each dictionary must include:
- "degree_level" (must be one of: "undergraduate", "graduate", "doctoral")
- "institution" (string)
- "field_of_study" (optional, string)
- "start_year" (optional, integer)
- "end_year" (optional, integer)

If no information is found, return an empty list `[]`.

Text:
{trunk}

{infobox_prompt}

Output JSON:"""
    return prompt


def validate_education_result(output: str) -> list[dict]:
    """
    Validate and clean LLM output that may contain markdown, extra text, or invalid JSON.

    Args:
        output (str): Raw LLM response.

    Returns:
        list[dict]: A validated list of education records.
    """
    def extract_json_block(text: str) -> str:
        # Try extracting markdown JSON block: ```json ... ```
        match = re.search(r"```json\s*(.*?)\s*```", text, re.DOTALL)
        if match:
            return match.group(1)
        
        # Try extracting any JSON array
        match = re.search(r"(\[\s*\{.*?\}\s*\])", text, re.DOTALL)
        if match:
            return match.group(1)

        # Try trimming text to start at first [
        idx = text.find("[")
        if idx != -1:
            return text[idx:]
        
        return text  # fallback

    raw_json = extract_json_block(output)

    try:
        data = json.loads(raw_json)
        if not isinstance(data, list):
            return []
    except json.JSONDecodeError:
        return []

    valid_degrees = {"undergraduate", "graduate", "doctoral"}
    cleaned = []

    for item in data:
        if not isinstance(item, dict):
            continue

        degree_level = item.get("degree_level")
        institution = item.get("institution")

        if degree_level not in valid_degrees or not institution:
            continue

        record = {
            "degree_level": degree_level,
            "institution": str(institution).strip(),
        }

        if "field_of_study" in item:
            record["field_of_study"] = str(item["field_of_study"]).strip()

        for year_field in ["start_year", "end_year"]:
            if year_field in item:
                try:
                    record[year_field] = int(item[year_field])
                except (ValueError, TypeError):
                    pass

        cleaned.append(record)

    return cleaned
    
text = """```json
[
    {
        "degree_level": "undergraduate",
        "institution": "Harvard University",
        "field_of_study": "Economics",
        "start_year": 1972,
        "end_year": 1976
    },
    {
        "degree_level": "graduate",
        "institution": "Harvard University",
        "field_of_study": "Economics",
        "start_year": 1976,
        "end_year": 1978
    },
    {
        "degree_level": "doctoral",
        "institution": "Harvard University",
        "field_of_study": "Economics",
        "start_year": 1978,
        "end_year": 1983
    },
    {
        "degree_level": "doctoral",
        "institution": "Stanford University",
        "field_of_study": "Medicine",
        "start_year": 1980,
        "end_year": 1983
    }
]
```"""

validate_education_result(text)

[{'degree_level': 'undergraduate',
  'institution': 'Harvard University',
  'field_of_study': 'Economics',
  'start_year': 1972,
  'end_year': 1976},
 {'degree_level': 'graduate',
  'institution': 'Harvard University',
  'field_of_study': 'Economics',
  'start_year': 1976,
  'end_year': 1978},
 {'degree_level': 'doctoral',
  'institution': 'Harvard University',
  'field_of_study': 'Economics',
  'start_year': 1978,
  'end_year': 1983},
 {'degree_level': 'doctoral',
  'institution': 'Stanford University',
  'field_of_study': 'Medicine',
  'start_year': 1980,
  'end_year': 1983}]

# 使用其 rank.unirank.net

In [3]:
database = pymongo.MongoClient(os.getenv("MONGO_URL"))['wikipedia_dump_v20250401']
page_complexity_stats = database['page_complexity_stats']
page_complexity = database['page_complexity']


# US

In [4]:
us_collages = []

# 从旧数据中获取大学排名 top rank
for i in range(1,6):
    json_path = f"data/us_{i}.json"
    with open(json_path, "r") as f:
        data = json.load(f)['data']['info']
        for title, item in data.items():
            us_collages.append({
                "title": title,
                "rank_global": item["rank_global"],
                "rank_country": item["rank_country"],
                "country": item["country_en"],
                "cnName": item["cnName"],
                "enName": item["endisplay"],
            })

# 对大学 title, id 验证
for item in us_collages:
    doc = page_complexity_stats.find_one({"title": item['title']})
    if not doc:
        doc = page_complexity_stats.find_one({"alias": item['title']})
        # print(item['title'],doc['title'])
        if not doc:
            print(item)

    if doc:
        item['title'] = doc['title']
        item['_id'] = doc['_id']

# UK collage

In [5]:
uk_collages = []

# 从旧数据中获取大学排名 top rank
for i in range(1,2):
    json_path = f"data/uk_{i}.json"
    with open(json_path, "r") as f:
        data = json.load(f)['data']['info']
        for title, item in data.items():
            uk_collages.append({
                "title": title,
                "rank_global": item["rank_global"],
                "rank_country": item["rank_country"],
                "country": item["country_en"],
                "cnName": item["cnName"],
                "enName": item["endisplay"],
            })

# 对大学 title, id 验证
for item in uk_collages:
    doc = page_complexity_stats.find_one({"title": item['title']})
    if not doc:
        doc = page_complexity_stats.find_one({"alias": item['title']})
        # print(item['title'],doc['title'])
        if not doc:
            print(item)

    if doc:
        item['title'] = doc['title']
        item['_id'] = doc['_id']

In [6]:
# 通过 infobox 找到 President
collage_list = us_collages + uk_collages
len(collage_list)


300

In [7]:
# 通过 infobox 找到 President

def get_internel_page(data):
    for item in data.get("links",[]):
        if item["type"] == "internal":
            return data["text"], item["page"] 
    return data["text"], None

except_count = 0
for item in collage_list:
    
    if "president" in item:
        continue
    
    doc = page_complexity.find_one({"_id":item["_id"]})
    
    infoboxes = doc.get("infoboxes") if doc else None
    if not infoboxes:
        print(item)
        continue
    text = None
    if "president" in infoboxes[0]:
        # print(infoboxes[0]["president"])
        text , president = get_internel_page(infoboxes[0]["president"])
        if president:
            item["president"] = president
            item["president_text"] = text
            continue
    if "chancellor" in infoboxes[0]:
        text , president = get_internel_page(infoboxes[0]["chancellor"])
        if president:
            item["president"] = president
            item["president_text"] = text
            continue
    if "superintendent" in infoboxes[0]:
        text , president = get_internel_page(infoboxes[0]["superintendent"])
        if president:
            item["president"] = president
            item["president_text"] = text
            continue
        
    if text:
        item["president_text"] = text
    except_count +=1
    # print(item, infoboxes)
print(except_count)
    # if infoboxes and not infobox_filter(infoboxes[0]):
    #     return None
    # if not doc:
    #     print(item)

52


In [8]:
#  补充 present page id
for item in collage_list:
    
    if "president" not in item:
        continue
    
    title = item['president']
    doc = page_complexity_stats.find_one({"title": title})
    if not doc:
        doc = page_complexity_stats.find_one({"alias": title})
        # print(item['title'],doc['title'])
        if not doc:
            print(item)
    if doc:
        item['president_id'] = doc['_id']
    

In [9]:
# 序列化数据, 以便于并发编程

collage_collection = database["collage_v20250426"]
# collage_collection.insert_many(collage_list)

In [15]:

from concurrent.futures import ThreadPoolExecutor, as_completed

def get_todo_docs():
    for doc in collage_collection.find({"president_id":{"$exists":True},
                                        "education_data": {"$exists": False}}):
        yield doc


def get_educations(infoboxes):
    for infoboxe_item in infoboxes:
        if "education" in infoboxe_item:
            # print(infoboxe_item["president"])
            # text , president = get_internel_page(infoboxe_item["president"])
            # if president:
            #     item["president"] = president
            #     item["president_text"] = text
            #     continue
            education = infoboxe_item["education"]['text']
            return f"education: {education}"
            # print(item['president'],":",education)
        if "alma_mater" in infoboxe_item:
            education = infoboxe_item["alma_mater"]['text']
            return f"alma mater: {education}"
     

model = "deepseek-r1-distill-llama-70b"  

def worker(collage_doc):
    person_doc = page_complexity.find_one({"_id": collage_doc['president_id']})
    if not person_doc:
        print("miss doc", person_doc)
        return
    
    plain_text = ""
    infobox_text = ""
    
    infoboxes = person_doc.get("infoboxes") if person_doc else None
    if infoboxes:
        infobox_text = get_educations(infoboxes)

    if infobox_text:
        # source
        person_doc['source_infobox'] = infobox_text
        
    education_result = []
    plain_text = person_doc.get("plaintext")
    infobox_text_flag = True
    
    LLMChat = AliyunChat(model=model, system_prompt=SYSTEM_PROMPT, max_tokens=4000)
    for trunk in split_plaintext_into_trunks(plain_text,400,20):
        prompt = generate_education_prompt(trunk, infobox_text if infobox_text_flag else "")
        infobox_text_flag = False
        LLMChat.resetChat()
        response = LLMChat.startChat(prompt)

        ret_data = validate_education_result(response)
        if len(response) > 15 and len(ret_data) == 0:
            print("response empty:", response)
        if ret_data:
            education_result.extend(ret_data)
    if len(education_result) > 0:
        upset_doc = {
            "education_data": education_result,
        }
        if infobox_text:
            upset_doc["source_infobox"] = infobox_text
        collage_collection.update_one(
            {"_id": collage_doc['_id']},
            {"$set": upset_doc},
            upsert=False
        )
        
max_workers = 40
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(worker, item) for item in get_todo_docs()]
    # for future in as_completed(futures):
    #     future.result()  # 等待任务完成，捕获异常
        
    for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
        future.result()  # 等待任务完成，捕获异常
    # break

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:55<00:00, 27.75s/it]


In [29]:
# 将数据写入 excel
import pandas as pd
doc_list = []
for doc in collage_collection.find():
    
    for item in doc.get("education_data", []):
        if item["degree_level"] == "undergraduate":
            for key, value in item.items():
                if key == "degree_level":
                    continue
                new_key = ""
                for i in range(6):
                    new_key = f"undergraduate_{key}_{i}"
                    if new_key not in doc:
                        break
                doc[new_key] = value
            
        if item["degree_level"] == "graduate":
            for key, value in item.items():
                if key == "degree_level":
                    continue
                new_key = ""
                for i in range(6):
                    new_key = f"graduate_{key}_{i}"
                    if new_key not in doc:
                        break
                doc[new_key] = value
            
        if item["degree_level"] == "doctoral":
            for key, value in item.items():
                if key == "degree_level":
                    continue
                new_key = ""
                for i in range(6):
                    new_key = f"doctoral_{key}_{i}"
                    if new_key not in doc:
                        break
                doc[new_key] = value
    
    doc_list.append(doc)
df = pd.DataFrame(doc_list)
df.to_excel("collage.xlsx", index=False)

# QS collage rank
因为 US collage 不足 200 个. 改用其他数据

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "QS World University Rankings 2025 (Top global universities).csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "melissamonfared/qs-world-university-rankings-2025",
  file_path,
  pandas_kwargs={"encoding": "latin1"}  # Specify the encoding to handle special characters
)

print("First 5 records:", df.head())

/tmp/ipykernel_2446165/1031589338.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:   RANK_2025 RANK_2024                              Institution_Name  \
0         1         1  Massachusetts Institute of Technology (MIT)    
1         2         6                       Imperial College London   
2         3         3                          University of Oxford   
3         4         4                            Harvard University   
4         5         2                       University of Cambridge   

         Location    Region SIZE FOCUS RES. STATUS  Academic_Reputation_Score  \
0   United States  Americas    M    CO   VH      B                      100.0   
1  United Kingdom    Europe    L    FC   VH      A                       98.5   
2  United Kingdom    Europe    L    FC   VH      A                      100.0   
3   United States  Americas    L    FC   VH      B                      100.0   
4  United Kingdom    Europe    L    FC   VH      A                      100.0   

   ... International_Faculty_Rank  International_Students_Score  \
0 

In [ ]:
write_key = ['RANK_2025','RANK_2024', 'Institution_Name', 'Location', 'Region','SIZE' ,'FOCUS','RES.', 'STATUS','Overall_Score']

In [ ]:
df[df["Location"]=="United States"][write_key].head(90)

,RANK_2025,RANK_2024,Institution_Name,Location,Region,SIZE,FOCUS,RES.,STATUS,Overall_Score
0,1,1,Massachusetts Institute of Technology (MIT),United States,Americas,M,CO,VH,B,100
3,4,4,Harvard University,United States,Americas,L,FC,VH,B,96.8
5,6,5,Stanford University,United States,Americas,L,FC,VH,B,96.1
9,10,15,California Institute of Technology (Caltech),United States,Americas,S,CO,VH,B,90.9
10,11,12,University of Pennsylvania,United States,Americas,L,FC,VH,B,90.3
...,...,...,...,...,...,...,...,...,...,...
607,601-610,494,Rensselaer Polytechnic Institute,United States,Americas,M,CO,VH,B,NaN
611,601-610,575,University of South Carolina,United States,Americas,XL,FC,VH,A,NaN
630,621-630,563,University of South Florida,United States,Americas,XL,CO,VH,A,NaN
634,631-640,631-640,Boston College,United States,Americas,L,FO,VH,B,NaN


In [ ]:
df[df["Location"]=="United Kingdom"].head(30)

,RANK_2025,RANK_2024,Institution_Name,Location,Region,SIZE,FOCUS,RES.,STATUS,Academic_Reputation_Score,...,International_Faculty_Rank,International_Students_Score,International_Students_Rank,International_Research_Network_Score,International_Research_Network_Rank,Employment_Outcomes_Score,Employment_Outcomes_Rank,Sustainability_Score,Sustainability_Rank,Overall_Score
1,2,6,Imperial College London,United Kingdom,Europe,L,FC,VH,A,98.5,...,66,99.6,44,97.4,34,93.4,61,99.7,6,98.5
2,3,3,University of Oxford,United Kingdom,Europe,L,FC,VH,A,100.0,...,120,97.7,73,100.0,1,100.0,3,85.0,126,96.9
4,5,2,University of Cambridge,United Kingdom,Europe,L,FC,VH,A,100.0,...,73,94.8,98,99.3,10,100.0,5,84.8,127=,96.7
8,9,9,UCL,United Kingdom,Europe,XL,FC,VH,A,99.5,...,106,100.0,21,99.9,2,70.3,157,74.8,190=,91.6
26,27,22,The University of Edinburgh,United Kingdom,Europe,XL,FC,VH,A,98.3,...,111,99.8,41,99.5,7,55.9,222,99.0,15=,83.3
34,34,32,The University of Manchester,United Kingdom,Europe,XL,FC,VH,A,95.6,...,169,99.2,55,99.2,12,87.6,90,99.8,3,82
39,40,40,King's College London,United Kingdom,Europe,XL,FC,VH,A,90.3,...,103,99.5,47,98.6,17,59.8,205,96.2,44=,80.2
51,50,45,The London School of Economics and Political S...,United Kingdom,Europe,L,SP,VH,A,67.2,...,40,100.0,12,81.2,305,99.6,15,91.3,85,76
53,54,55,University of Bristol,United Kingdom,Europe,L,FC,VH,A,75.8,...,206,89.8,121,94.6,84,51.2,250,98.9,17,75.4
68,69,67,The University of Warwick,United Kingdom,Europe,L,FC,VH,A,72.2,...,119,99.1,57,94.2,92,65.7,175,89.7,98,68.2
